In [ ]:
import numpy as np
import numpy
import matplotlib.pyplot as plt
import sys
    

def phantom( t, params ):

    a = params[0]
    n = params[1]

    if t==0:
        # Imagem
        f = numpy.loadtxt('furu.dat')[0:692:8,0:692:8]

    elif t==1:

        # norma-2 (circulo)
        
        x = numpy.linspace(-a,a,n)    
        xx,yy = numpy.meshgrid(x,x)

        f = (xx**2 + yy**2 < 0.3**2).astype(numpy.double)

    elif t==2:

        # norma-1 (losango)
        
        x = numpy.linspace(-a,a,n)    
        xx,yy = numpy.meshgrid(x,x)

        f = ( numpy.abs(xx) + numpy.abs(yy) < 0.3 ).astype(numpy.double)

    else:

        # norma-p ( `quadrado`, p->infty)
        
        p = 10
        
        x = numpy.linspace(-a,a,n)    
        xx,yy = numpy.meshgrid(x,x)

        f = ( numpy.abs(xx)**p + numpy.abs(yy)**p < 0.3**p ).astype(numpy.double)
        
        
    return f

In [ ]:
def radon( img, params ) :

    a = params[0]
    R = params[1]
    V = params[2]
    initial = params[3] # initial angle
    final = params[4]   # final angle
    
    t  = numpy.linspace(-a,a, R)
    theta = numpy.linspace( (numpy.pi * initial / 180.0), (numpy.pi * final / 180.0), V, endpoint=False) 
    s  = numpy.linspace(-a,a,R)

    sino = numpy.zeros([R,V])

    dx = 2.*a/(R-1)
    dy = dx
    
    for i in range(R):

        for j in range(V):
            
            for k in range(R):

                x = t[i] * numpy.cos(theta[j]) - s[k] * numpy.sin(theta[j])
                y = t[i] * numpy.sin(theta[j]) + s[k] * numpy.cos(theta[j])
        
                ix = ( numpy.ceil( (x + a)/dx) ).astype(numpy.int)
                iy = ( numpy.ceil( (y + a)/dy) ).astype(numpy.int)

                if ((ix > 0) & (ix < R) & (iy >0) & (iy < R)):         
                    sino[i][j] += img[ix][iy]
                    
    return sino

In [ ]:
#

def backp ( sino, params ):

    a = params[0]
    R = params[1]
    V = params[2]
    
    th = numpy.linspace((numpy.pi * params[3] / 180.0), (numpy.pi * params[4] / 180.0), V, endpoint=False)
    
    x = numpy.linspace(-a,a, R)
    y = x

    dt = (2*a)/(R-1)

    b = numpy.zeros([R,R])
    
    for i in range(R):
        for j in range(R):

            cumsum = 0

            for k in range(V):

                t = x[i] * numpy.cos(th[k]) + y[j] * numpy.sin(th[k])

                idx = numpy.ceil((t + a)/dt).astype(numpy.int) 

                if ((idx > 0) & (idx < R)):
                    cumsum += sino[idx][k]

            b[i][j] = cumsum

    return b

#

In [ ]:
def filter( sino, params ):

    a = params[0]
    R = params[1]
    V = params[2]
    
    th = numpy.linspace((numpy.pi * params[3] / 180.0), (numpy.pi * params[4] / 180.0), V, endpoint=False)
    t  = numpy.linspace(-a, a, R)
    
    dt = (2*a)/(R-1)
    
    wc = 1.0/(2*dt)
    
    w = numpy.linspace(-wc, wc, R)
    
    h = numpy.abs(w)
    
    G = numpy.fft.fftshift(numpy.transpose(numpy.kron(numpy.ones((V, 1)), h)))
    
    B = numpy.fft.fft(sino, axis=0)
    
    C = B * G
#     C = B
    
    #---
    
    D = numpy.fft.ifft(C, axis=0)
    
    return D.real 

In [ ]:
#
def volRender(vol):
    
    from mayavi import mlab
    mlab.init_notebook()
    
    mlab.figure(bgcolor=(1, 1, 1))
    
    mlab.pipeline.volume(mlab.pipeline.scalar_field(vol), vmin=0, vmax=vol.max().max())
    
    mlab.outline()
    
    mlab.show()

In [ ]:
def volContour(vol):
    
    from mayavi import mlab
    
    mlab.init_notebook()
    
    mlab.figure(bgcolor=(1, 1, 1))
    
    mlab.pipeline.image_plane_widget(mlab.pipeline.scalar_field(vol),plane_orientation='x_axes',slice_index=10,)
    
    mlab.pipeline.image_plane_widget(mlab.pipeline.scalar_field(vol),plane_orientation='y_axes',slice_index=10,)
    
    mlab.outline()
    mlab.show()

In [ ]:
#######

'''
def phantom3D( params ):

    a = params[0]
    R = params[1]

    x = numpy.linspace()

'''
############

In [ ]:
def tarugo( params  ):

    a = params[0]
    N = params[1]

    x = numpy.linspace(-1,1,N)
    xx,yy,zz = numpy.meshgrid(x,x,x)
    
    sphere = (xx**2 + zz**2 + yy**2 <  0.4**2).astype(numpy.double)

    cld = (xx**2 + yy**2 < 0.7**2).astype(numpy.double)
    
    vol = sphere + cld

    return vol    


In [ ]:
def projection( volume) :

    img = numpy.zeros([volume.shape[0],volume.shape[1]])
    
    for z in range(volume.shape[0]):
        
        img += volume[:][z][:]

    return img.T
    

####

In [ ]:
# %matplotlib notebook
# from mpl_toolkits.mplot3d import Axes3D

# params = ( 1.0, 74, 1, 90, 90)

# treisd = tarugo( params)

# # volContour( treisd )

# plt.imshow( treisd[37][:][:]  )
# plt.show()

# ##


# plt.imshow( projection( treisd ) )
# plt.show()



params = ( 1.0, 84, 180, 0, 180)
f = phantom( 0, params )

s = radon( f, params )

# plt.title('Projection signal')
# plt.plot(s)
# plt.show()


h = filter(s, params)

b = backp( h, params )

plt.figure(0)
plt.imshow(f)

plt.figure(1)
plt.imshow(s)

plt.figure(2)
plt.imshow(h)

plt.figure(3)
plt.imshow(b)

plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D, art3d  # NOQA
from matplotlib.cbook import _backports
from collections import defaultdict
import types

def voxels(self, *args, **kwargs):

    if len(args) >= 3:
        # underscores indicate position only
        def voxels(__x, __y, __z, filled, **kwargs):
            return (__x, __y, __z), filled, kwargs
    else:
        def voxels(filled, **kwargs):
            return None, filled, kwargs

    xyz, filled, kwargs = voxels(*args, **kwargs)

    # check dimensions
    if filled.ndim != 3:
        raise ValueError("Argument filled must be 3-dimensional")
    size = np.array(filled.shape, dtype=np.intp)

    # check xyz coordinates, which are one larger than the filled shape
    coord_shape = tuple(size + 1)
    if xyz is None:
        x, y, z = np.indices(coord_shape)
    else:
        x, y, z = (_backports.broadcast_to(c, coord_shape) for c in xyz)

    def _broadcast_color_arg(color, name):
        if np.ndim(color) in (0, 1):
            # single color, like "red" or [1, 0, 0]
            return _backports.broadcast_to(
                color, filled.shape + np.shape(color))
        elif np.ndim(color) in (3, 4):
            # 3D array of strings, or 4D array with last axis rgb
            if np.shape(color)[:3] != filled.shape:
                raise ValueError(
                    "When multidimensional, {} must match the shape of "
                    "filled".format(name))
            return color
        else:
            raise ValueError("Invalid {} argument".format(name))

    # intercept the facecolors, handling defaults and broacasting
    facecolors = kwargs.pop('facecolors', None)
    if facecolors is None:
        facecolors = self._get_patches_for_fill.get_next_color()
    facecolors = _broadcast_color_arg(facecolors, 'facecolors')

    # broadcast but no default on edgecolors
    edgecolors = kwargs.pop('edgecolors', None)
    edgecolors = _broadcast_color_arg(edgecolors, 'edgecolors')

    # include possibly occluded internal faces or not
    internal_faces = kwargs.pop('internal_faces', False)

    # always scale to the full array, even if the data is only in the center
    self.auto_scale_xyz(x, y, z)

    # points lying on corners of a square
    square = np.array([
        [0, 0, 0],
        [0, 1, 0],
        [1, 1, 0],
        [1, 0, 0]
    ], dtype=np.intp)

    voxel_faces = defaultdict(list)

    def permutation_matrices(n):
        """ Generator of cyclic permutation matices """
        mat = np.eye(n, dtype=np.intp)
        for i in range(n):
            yield mat
            mat = np.roll(mat, 1, axis=0)

    for permute in permutation_matrices(3):
        pc, qc, rc = permute.T.dot(size)
        pinds = np.arange(pc)
        qinds = np.arange(qc)
        rinds = np.arange(rc)

        square_rot = square.dot(permute.T)

        for p in pinds:
            for q in qinds:
                p0 = permute.dot([p, q, 0])
                i0 = tuple(p0)
                if filled[i0]:
                    voxel_faces[i0].append(p0 + square_rot)

                # draw middle faces
                for r1, r2 in zip(rinds[:-1], rinds[1:]):
                    p1 = permute.dot([p, q, r1])
                    p2 = permute.dot([p, q, r2])
                    i1 = tuple(p1)
                    i2 = tuple(p2)
                    if filled[i1] and (internal_faces or not filled[i2]):
                        voxel_faces[i1].append(p2 + square_rot)
                    elif (internal_faces or not filled[i1]) and filled[i2]:
                        voxel_faces[i2].append(p2 + square_rot)

                # draw upper faces
                pk = permute.dot([p, q, rc-1])
                pk2 = permute.dot([p, q, rc])
                ik = tuple(pk)
                if filled[ik]:
                    voxel_faces[ik].append(pk2 + square_rot)

    # iterate over the faces, and generate a Poly3DCollection for each voxel
    polygons = {}
    for coord, faces_inds in voxel_faces.items():
        # convert indices into 3D positions
        if xyz is None:
            faces = faces_inds
        else:
            faces = []
            for face_inds in faces_inds:
                ind = face_inds[:, 0], face_inds[:, 1], face_inds[:, 2]
                face = np.empty(face_inds.shape)
                face[:, 0] = x[ind]
                face[:, 1] = y[ind]
                face[:, 2] = z[ind]
                faces.append(face)

        poly = art3d.Poly3DCollection(faces,
            facecolors=facecolors[coord],
            edgecolors=edgecolors[coord],
            **kwargs
        )
        self.add_collection3d(poly)
        polygons[coord] = poly

    return polygons

In [ ]:
alpha = .013

aux = numpy.empty((74, 74, 74, 4))

for i, v in enumerate(treisd):
    for j, x in enumerate(v):
        for k, y in enumerate(x):
            if y == 1.0:
                aux[i][j][k] = [0, 1, 0, alpha]
            elif y == 2.0:
                aux[i][j][k] = [1, 1, 0, alpha]

# aux = numpy.array(aux)
                

In [ ]:
fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
ax = Axes3D(fig)
ax.voxels = types.MethodType(voxels, ax)
ax.voxels(treisd, facecolors=aux, internal_faces=True)
# plt.plot(treisd[0])
plt.show()
# plt.imshow(aux[37,:,:])

In [ ]:
# aux[0][0][0] = [1, 0, 0, alpha]
print(type(aux[0][0]))
print(treisd.shape)

In [ ]:
spatial_axes = [5, 5, 5]
filled = np.ones(spatial_axes, dtype=np.bool)

colors = np.empty(spatial_axes + [4], dtype=np.float32)
alpha = .5
colors[0] = [1, 0, 0, alpha]
colors[1] = [0, 1, 0, alpha]
colors[2] = [0, 0, 1, alpha]
colors[3] = [1, 1, 0, alpha]
colors[4] = [0, 1, 1, alpha]
print(colors.shape)

In [ ]:
# %matplotlib notebook

fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
ax = Axes3D(fig)
ax.voxels(treisd)
# plt.plot(treisd[0])
plt.show()


In [ ]:
from mayavi import mlab

mlab.init_notebook()

s = mlab.test_plot3d()

s

In [ ]:
%matplotlib inline  

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D



# What follows is a copy of the 3D plot example code.
# Data is randomly generated so there is no external data import.

def randrange(n, vmin, vmax):
    return (vmax-vmin)*np.random.rand(n) + vmin

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100
for c, m, zl, zh in [('r', 'o', -60, -25), ('b', '^', -30, -5)]:
    xs = randrange(n, 23, 50)
    ys = randrange(n, 0, 100)
    zs = randrange(n, zl, zh)
    ax.scatter(xs, ys, zs, c=c, marker=m)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')
theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
z = np.linspace(-2, 2, 100)
r = z**2 + 1
x = r * np.sin(theta)
y = r * np.cos(theta)
ax.plot(x, y, z, label='parametric curve')
ax.legend()

plt.show()